In [1]:
# Load Spark engine
import findspark
findspark.init()

In [2]:
import sys
import re
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext

In [3]:
sc = SparkContext(master="local[2]", appName="firstapplication")
sc.setLogLevel("ERROR")
ssc = StreamingContext(sc, 1)
ssc.checkpoint("checkpoint")

In [4]:
lines = ssc.socketTextStream("localhost", 8000)
# open cmd and tylinpe: nc -lk 7000

In [5]:
#remove the numbers from the streamed data.
def remove_numbers(word):
    result = ''.join([i for i in word if not i.isdigit()])
    return result

In [6]:
#remove all the symbols
def remove_symbols(word):
    result1 = re.sub(r'[^a-zA-Z_\s]+', '', word)
    return result1

In [7]:
#remove white spaces
def remove_spaces(word):
    result2 = word.strip()
    return result2

In [8]:
#remove duplicates
def remove_duplicates(word):
    
    word = ''.join(list(dict.fromkeys(list(word))))
    return word

In [9]:
#classify data based on the word length
def text(word):
    if len(word)>0 :
        return len(word)
    else :
        return 0
        
    

In [10]:
words = lines.flatMap(lambda line: line.split(" "))
words = words.map(lambda word:remove_numbers(word))
words = words.map(lambda word:remove_symbols(word))
words = words.map(lambda word:remove_spaces(word))
words = words.map(lambda word:remove_duplicates(word))
words = words.map(lambda word:text(word))
words = words.reduce(lambda a,b: a+b)


#words.countByValueAndWindow(10, 5).pprint()
words.window(10, 5).pprint()

In [11]:
ssc.start()

-------------------------------------------
Time: 2022-09-09 14:40:53
-------------------------------------------

-------------------------------------------
Time: 2022-09-09 14:40:58
-------------------------------------------

-------------------------------------------
Time: 2022-09-09 14:41:03
-------------------------------------------
8

-------------------------------------------
Time: 2022-09-09 14:41:08
-------------------------------------------
8
10

-------------------------------------------
Time: 2022-09-09 14:41:13
-------------------------------------------
10
4

-------------------------------------------
Time: 2022-09-09 14:41:18
-------------------------------------------
4

-------------------------------------------
Time: 2022-09-09 14:41:23
-------------------------------------------

-------------------------------------------
Time: 2022-09-09 14:41:28
-------------------------------------------
7

-------------------------------------------
Time: 2022-09-09 14:

In [12]:
# Stop the Streaming session
ssc.stop(stopSparkContext=True, stopGraceFully=True)